# Use pretrained model for test predictions

In [2]:
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.6/226.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 17.5 MB/s eta 0:00:00
  Installing build dep

In [3]:
from huggingface_hub import login
from utils import WRITE_ACCESS_TOKEN

login(WRITE_ACCESS_TOKEN)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import os
import torch
import pandas as pd
from evaluate import load

from load_fleurs_nl import load_fleurs_nl
from load_fleurs_zu import load_fleurs_zu

from datasets import Audio, load_dataset
from utils import SR, remove_special_characters_batch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ProcessorWithLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def predict_transcription(audio_sample, model, processor):
    # Get model inputs
    inputs = processor(
        audio_sample["audio"]["array"],
        sampling_rate=audio_sample["audio"]["sampling_rate"],
        return_tensors="pt",
        padding=True,
    ).to(device)

    # Pass inputs into model to get logits
    with torch.no_grad():
        logits = model(**inputs).logits

    # Decode logits to get predicted transcription
    if isinstance(processor, Wav2Vec2ProcessorWithLM):
        pred = processor.batch_decode(logits.cpu().numpy()).text
        pred = pred[0].lower()
    else:
        predicted_ids = torch.argmax(logits, dim=-1)
        pred = processor.batch_decode(predicted_ids)
        pred = pred[0].lower()

    return pred

### Basic fine-tuning models

In [ ]:
for dataset_name, repo_name in [
    ("asr_af", "lucas-meyer/xls-r-asr_af-run1"),
    ("asr_af", "lucas-meyer/xls-r-asr_af-run2"),
    ("asr_af", "lucas-meyer/xls-r-asr_af-run3"),
    ("asr_af", "lucas-meyer/xls-r-asr_af-run4"),
    ("asr_af", "lucas-meyer/xls-r-asr_af-run5"),
    ("asr_af", "lucas-meyer/xls-r-asr_af-run6"),
    ("asr_af", "lucas-meyer/xls-r-asr_af-run7"),
    ("asr_af", "lucas-meyer/xls-r-asr_af-run8"),

    ("asr_xh", "lucas-meyer/xls-r-asr_xh-run1"),
    ("asr_xh", "lucas-meyer/xls-r-asr_xh-run2"),
    ("asr_xh", "lucas-meyer/xls-r-asr_xh-run3"),
    ("asr_xh", "lucas-meyer/xls-r-asr_xh-run4"),
    ("asr_xh", "lucas-meyer/xls-r-asr_xh-run5"),
    ("asr_xh", "lucas-meyer/xls-r-asr_xh-run6"),
    ("asr_xh", "lucas-meyer/xls-r-asr_xh-run7"),
    ("asr_xh", "lucas-meyer/xls-r-asr_xh-run8"),

#     ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run8"), # ignore this lol
#     ("asr_af", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run8"), # ignore this lol
]:
    # DL data
    if dataset_name == "asr_af" or dataset_name == "asr_xh":
        val_set = load_dataset("lucas-meyer/" + dataset_name, split="validation")
        test_set = load_dataset("lucas-meyer/" + dataset_name, split="test")
        val_set = val_set.cast_column("audio", Audio(sampling_rate=SR))
        test_set = test_set.cast_column("audio", Audio(sampling_rate=SR))
    else:
        dataset_dir = os.path.join("data", "speech_data", dataset_name)
        if not os.path.exists(dataset_dir):
            os.makedirs(dataset_dir, exist_ok=True)
            # Create dataset by combining 3 datasets into an audiofolder
            csv_entries = []
            if (dataset_name == "fleurs_nl"):
                csv_entries += load_fleurs_nl(write_audio=True)
            elif (dataset_name == "fleurs_zu"):
                csv_entries += load_fleurs_zu(write_audio=True)
            metadata = pd.DataFrame(csv_entries, columns=['file_name', 'transcription'])
            metadata.to_csv(path_or_buf=os.path.join(dataset_dir, "metadata.csv"), sep=",", index=False)

            # Load dataset from audiofolder that you created
            dataset = load_dataset("audiofolder", data_dir=dataset_dir)
        else:
            # Load dataset from audiofolder that you created
            dataset = load_dataset("audiofolder", data_dir=dataset_dir)

        # Downsample audio to SR = 16000 and init train/val/test sets
        val_set = dataset["validation"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")
        test_set = dataset["test"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")
        val_set = val_set.map(remove_special_characters_batch)
        test_set = test_set.map(remove_special_characters_batch)
        # torch.cuda.empty_cache()

    # DL model
    model_basic = Wav2Vec2ForCTC.from_pretrained(repo_name).to(device)
    processor_basic = Wav2Vec2Processor.from_pretrained(repo_name)

    print(f"Results: {repo_name}", end="\n\n")

    # --------------------------------------------------------------
    # VALIDATION SET SCORE
    # --------------------------------------------------------------
    true_transcriptions = []
    model_predictions = []
    model_with_LM_predictions = []

    for i in range(len(val_set)):
        pred_basic = predict_transcription(val_set[i], model_basic, processor_basic)
        model_predictions.append(pred_basic)
        if "fleurs" in dataset_name:
            true_transcriptions.append(val_set[i]["sentence"].lower())
        else:
            true_transcriptions.append(val_set[i]["transcription"].lower())

        # Print progress
        print(f"\r{i+1}/{len(val_set)}\t\t", end="")
    print("")

    wer = load("wer")
    wer_score_model = wer.compute(predictions=model_predictions, references=true_transcriptions)
    print(f"Validation score: {wer_score_model}", end="\n\n")

    # --------------------------------------------------------------
    # TEST SET SCORE
    # --------------------------------------------------------------
    true_transcriptions = []
    model_predictions = []
    model_with_LM_predictions = []

    for i in range(len(test_set)):
        pred_basic = predict_transcription(test_set[i], model_basic, processor_basic)
        model_predictions.append(pred_basic)
        if "fleurs" in dataset_name:
            true_transcriptions.append(test_set[i]["sentence"].lower())
        else:
            true_transcriptions.append(test_set[i]["transcription"].lower())

        # Print progress
        print(f"\r{i+1}/{len(test_set)}\t\t", end="")
    print("")

    wer = load("wer")
    wer_score_model = wer.compute(predictions=model_predictions, references=true_transcriptions)
    print(f"Test score: {wer_score_model}", end="\n\n")

Results: lucas-meyer/xls-r-asr_af-run1

447/447		


Validation score: 0.4245732738735968

476/476		
Test score: 0.4331452565280147



Results: lucas-meyer/xls-r-asr_af-run2

447/447		
Validation score: 0.42472704905428266

476/476		
Test score: 0.4341949875344443



Results: lucas-meyer/xls-r-asr_af-run3

447/447		
Validation score: 0.43057050592034446

476/476		
Test score: 0.43655688229891093



Results: lucas-meyer/xls-r-asr_af-run4

447/447		
Validation score: 0.3789020452099031

476/476		
Test score: 0.3840703319774308



...

Results: lucas-meyer/xls-r-asr_xh-run6

338/338		
Validation score: 0.5306122448979592

627/627		
Test score: 0.5387755102040817



Results: lucas-meyer/xls-r-asr_xh-run7

338/338		
Validation score: 0.5513443472627146

627/627		
Test score: 0.5210884353741496



Results: lucas-meyer/xls-r-asr_xh-run8

338/338		
Validation score: 0.5387107223841918

627/627		
Test score: 0.5371126228269085



Results: lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run8

338/338		
Validation score: 1.1956592160673793

627/627		
Test score: 1.127891156462585



Results: lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run8

447/447		
Validation score: 0.9675534368752883

476/476		
Test score: 0.9813672746358746



### Sequential fine-tuning models

In [3]:
for dataset_name, repo_name in [
    ("asr_af", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run1"),
    ("asr_af", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run2"),
    ("asr_af", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run3"),
    ("asr_af", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run4"),
    ("asr_af", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run5"),
    ("asr_af", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run6"),
    ("asr_af", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run7"),
    ("asr_af", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run8"),

    ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run1"),
    ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run2"),
    ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run3"),
    ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run4"),
    ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run5"),
    ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run6"),
    ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run7"),
    ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run8"),
]:
    # DL data
    if dataset_name == "asr_af" or dataset_name == "asr_xh":
        val_set = load_dataset("lucas-meyer/" + dataset_name, split="validation")
        test_set = load_dataset("lucas-meyer/" + dataset_name, split="test")
        val_set = val_set.cast_column("audio", Audio(sampling_rate=SR))
        test_set = test_set.cast_column("audio", Audio(sampling_rate=SR))
    else:
        dataset_dir = os.path.join("data", "speech_data", dataset_name)
        if not os.path.exists(dataset_dir):
            os.makedirs(dataset_dir, exist_ok=True)
            # Create dataset by combining 3 datasets into an audiofolder
            csv_entries = []
            if (dataset_name == "fleurs_nl"):
                csv_entries += load_fleurs_nl(write_audio=True)
            elif (dataset_name == "fleurs_zu"):
                csv_entries += load_fleurs_zu(write_audio=True)
            metadata = pd.DataFrame(csv_entries, columns=['file_name', 'transcription'])
            metadata.to_csv(path_or_buf=os.path.join(dataset_dir, "metadata.csv"), sep=",", index=False)

            # Load dataset from audiofolder that you created
            dataset = load_dataset("audiofolder", data_dir=dataset_dir)
        else:
            # Load dataset from audiofolder that you created
            dataset = load_dataset("audiofolder", data_dir=dataset_dir)

        # Downsample audio to SR = 16000 and init train/val/test sets
        val_set = dataset["validation"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")
        test_set = dataset["test"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")
        val_set = val_set.map(remove_special_characters_batch)
        test_set = test_set.map(remove_special_characters_batch)
        # torch.cuda.empty_cache()

    # DL model
    model_basic = Wav2Vec2ForCTC.from_pretrained(repo_name).to(device)
    processor_basic = Wav2Vec2Processor.from_pretrained(repo_name)

    print(f"Results: {repo_name}", end="\n\n")

    # --------------------------------------------------------------
    # VALIDATION SET SCORE
    # --------------------------------------------------------------
    true_transcriptions = []
    model_predictions = []
    model_with_LM_predictions = []

    for i in range(len(val_set)):
        pred_basic = predict_transcription(val_set[i], model_basic, processor_basic)
        model_predictions.append(pred_basic)
        if "fleurs" in dataset_name:
            true_transcriptions.append(val_set[i]["sentence"].lower())
        else:
            true_transcriptions.append(val_set[i]["transcription"].lower())

        # Print progress
        print(f"\r{i+1}/{len(val_set)}\t\t", end="")
    print("")

    wer = load("wer")
    wer_score_model = wer.compute(predictions=model_predictions, references=true_transcriptions)
    print(f"Validation score: {wer_score_model}", end="\n\n")

    # --------------------------------------------------------------
    # TEST SET SCORE
    # --------------------------------------------------------------
    true_transcriptions = []
    model_predictions = []
    model_with_LM_predictions = []

    for i in range(len(test_set)):
        pred_basic = predict_transcription(test_set[i], model_basic, processor_basic)
        model_predictions.append(pred_basic)
        if "fleurs" in dataset_name:
            true_transcriptions.append(test_set[i]["sentence"].lower())
        else:
            true_transcriptions.append(test_set[i]["transcription"].lower())

        # Print progress
        print(f"\r{i+1}/{len(test_set)}\t\t", end="")
    print("")

    wer = load("wer")
    wer_score_model = wer.compute(predictions=model_predictions, references=true_transcriptions)
    print(f"Test score: {wer_score_model}", end="\n\n")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/476 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/2723 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/447 [00:00<?, ? examples/s]

Results: lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run1

447/447		


...

Results: lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run4

338/338		
Validation score: 0.540006478781989

627/627		
Test score: 0.5517762660619804



Results: lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run5

338/338		
Validation score: 0.5578231292517006

627/627		
Test score: 0.5765684051398338



Results: lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run6

338/338		
Validation score: 0.5221898283122773

627/627		
Test score: 0.5529856386999245



Results: lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run7

338/338		
Validation score: 0.5011337868480725

627/627		
Test score: 0.4988662131519274



Results: lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run8

338/338		
Validation score: 0.5364431486880467

627/627		
Test score: 0.5685563114134543



### Best models (with LMs)

In [ ]:
for dataset_name, repo_name in [
    ("asr_af", "lucas-meyer/xls-r-asr_af-run8-with-LM"),
    ("asr_af", "lucas-meyer/seq-xls-r-fleurs_nl-run2-asr_af-run6-with-LM"),
    ("asr_xh", "lucas-meyer/xls-r-asr_xh-run3-with-LM"),
    ("asr_xh", "lucas-meyer/seq-xls-r-fleurs_zu-run3-asr_xh-run7-with-LM"),
]:
    # DL data
    if dataset_name == "asr_af" or dataset_name == "asr_xh":
        val_set = load_dataset("lucas-meyer/" + dataset_name, split="validation")
        test_set = load_dataset("lucas-meyer/" + dataset_name, split="test")
        val_set = val_set.cast_column("audio", Audio(sampling_rate=SR))
        test_set = test_set.cast_column("audio", Audio(sampling_rate=SR))
    else:
        dataset_dir = os.path.join("data", "speech_data", dataset_name)
        if not os.path.exists(dataset_dir):
            os.makedirs(dataset_dir, exist_ok=True)
            # Create dataset by combining 3 datasets into an audiofolder
            csv_entries = []
            if (dataset_name == "fleurs_nl"):
                csv_entries += load_fleurs_nl(write_audio=True)
            elif (dataset_name == "fleurs_zu"):
                csv_entries += load_fleurs_zu(write_audio=True)
            metadata = pd.DataFrame(csv_entries, columns=['file_name', 'transcription'])
            metadata.to_csv(path_or_buf=os.path.join(dataset_dir, "metadata.csv"), sep=",", index=False)

            # Load dataset from audiofolder that you created
            dataset = load_dataset("audiofolder", data_dir=dataset_dir)
        else:
            # Load dataset from audiofolder that you created
            dataset = load_dataset("audiofolder", data_dir=dataset_dir)

        # Downsample audio to SR = 16000 and init train/val/test sets
        val_set = dataset["validation"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")
        test_set = dataset["test"].cast_column("audio", Audio(sampling_rate=SR)).rename_column("transcription", "sentence")
        val_set = val_set.map(remove_special_characters_batch)
        test_set = test_set.map(remove_special_characters_batch)
        # torch.cuda.empty_cache()

    # DL model
    model_with_LM = Wav2Vec2ForCTC.from_pretrained(repo_name).to(device)
    processor_with_LM = Wav2Vec2ProcessorWithLM.from_pretrained(repo_name)

    print(f"Results: {repo_name}", end="\n\n")

    # --------------------------------------------------------------
    # VALIDATION SET SCORE
    # --------------------------------------------------------------
    true_transcriptions = []
    model_with_LM_predictions = []

    for i in range(len(val_set)):
        pred_with_LM = predict_transcription(val_set[i], model_with_LM, processor_with_LM)
        model_with_LM_predictions.append(pred_with_LM)
        if "fleurs" in dataset_name:
            true_transcriptions.append(val_set[i]["sentence"].lower())
        else:
            true_transcriptions.append(val_set[i]["transcription"].lower())

        # Print progress
        print(f"\r{i+1}/{len(val_set)}\t\t", end="")
    print("")

    wer = load("wer")
    wer_score_model_with_LM = wer.compute(predictions=model_with_LM_predictions, references=true_transcriptions)
    print(f"Validation score: {wer_score_model_with_LM}", end="\n\n")

    # --------------------------------------------------------------
    # TEST SET SCORE
    # --------------------------------------------------------------
    true_transcriptions = []
    model_with_LM_predictions = []

    for i in range(len(test_set)):
        pred_with_LM = predict_transcription(test_set[i], model_with_LM, processor_with_LM)
        model_with_LM_predictions.append(pred_with_LM)
        if "fleurs" in dataset_name:
            true_transcriptions.append(test_set[i]["sentence"].lower())
        else:
            true_transcriptions.append(test_set[i]["transcription"].lower())

        # Print progress
        print(f"\r{i+1}/{len(test_set)}\t\t", end="")
    print("")

    wer = load("wer")
    wer_score_model_with_LM = wer.compute(predictions=model_with_LM_predictions, references=true_transcriptions)
    print(f"Test score: {wer_score_model_with_LM}", end="\n\n")